In [ ]:
def G(x, mean, std):
    return np.exp(-0.5*np.square((x-mean)/std))

def ExcesiveDark(x, M):
    return G(x, -25, M/9)

def VeryDark(x, M):
    return G(x, 0, M/9)

def Dark(x, M):
    return G(x, M/2, M/9)

def ModeratelyDark(x, M):
    return G(x, 5*M/4, M/9)

def ModeratelyBright(x, M):
    return G(x, M+(255-M)/9, (255-M)/9)

def Bright(x, M):
    return G(x, M+(255-M)/2, (255-M)/9)

def VeryBright(x, M):
    return G(x, 255, (255-M)/9)

def ExtremelyBright(x, M):
    return G(x, 300, (255-M)/9)

In [ ]:
for M in (128, 64, 192):
    x = np.arange(-50, 306)
    
    ED = ExtremelyDark(x, M)
    VD = VeryDark(x, M)
    Da = Dark(x, M)
    SD = MediumDark(x, M)
    SB = MediumBright(x, M)
    Br = Bright(x, M)
    VB = VeryBright(x, M)
    EB = ExtremelyBright(x, M)
    
def OutputFuzzySet(x, f, M, thres):
    x = np.array(x)
    result = f(x, M)
    result[result > thres] = thres
    return result

def AggregateFuzzySets(fuzzy_sets):
    return np.max(np.stack(fuzzy_sets), axis=0)

def Infer(i, M, get_fuzzy_set=False):
    # Calculate degree of membership for each class
    VD = VeryDark(i, M)
    Da = Dark(i, M)
    SD = MediumDark(i, M)
    SB = MediumBright(i, M)
    Br = Bright(i, M)
    VB = VeryBright(i, M)
    
    
    x = np.arange(-50, 306)
    Inferences = (
        OutputFuzzySet(x, ExtremelyDark, M, VD),
        OutputFuzzySet(x, VeryDark, M, Da),
        OutputFuzzySet(x, Dark, M, SD),
        OutputFuzzySet(x, Bright, M, SB),
        OutputFuzzySet(x, VeryBright, M, Br),
        OutputFuzzySet(x, ExtremelyBright, M, VB)
    )
    
    
    fuzzy_output = AggregateFuzzySets(Inferences)
    
    
    if get_fuzzy_set:
        return np.average(x, weights=fuzzy_output), fuzzy_output
    return np.average(x, weights=fuzzy_output)

In [ ]:
def FuzzyContrastEnhance(rgb):
    
    lab = cv2.cvtColor(rgb, cv2.COLOR_RGB2LAB)
    l = lab[:, :, 0]
       
    M = np.mean(l)
    if M < 128:
        M = 127 - (127 - M)/2
    else:
        M = 128 + M/2
           
    x = list(range(-50,306))
    FuzzyTransform = dict(zip(x,[Infer(np.array([i]), M) for i in x]))
    
    
    u, inv = np.unique(l, return_inverse = True)
    l = np.array([FuzzyTransform[i] for i in u])[inv].reshape(l.shape)
        
    Min = np.min(l)
    Max = np.max(l)
    lab[:, :, 0] = (l - Min)/(Max - Min) * 255
        
    return cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

def HE(rgb):
    lab = cv2.cvtColor(rgb, cv2.COLOR_RGB2LAB)
    lab[:, :, 0] = cv2.equalizeHist(lab[:, :, 0])
    return cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

def CLAHE(rgb):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    lab = cv2.cvtColor(rgb, cv2.COLOR_RGB2LAB)
    lab[:, :, 0] = clahe.apply(lab[:, :, 0])
    return cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

In [ ]:
class GlobalAvgPool2d(nn.Module):
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()

    def forward(self, x):
        N = x.data.size(0)
        C = x.data.size(1)
        H = x.data.size(2)
        W = x.data.size(3)
        x = F.avg_pool2d(x, (H, W))
        x = x.view(N, C)
        return x


class Darknet19(BaseModel):
    def __init__(self, pretrained=True):
        super(Darknet19, self).__init__()

        self.features = nn.Sequential(OrderedDict([
        ('layer1', nn.Sequential(OrderedDict([
            ('conv1_1', nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn1_1', nn.BatchNorm2d(32)),
            ('leaky1_1', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool1', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer2', nn.Sequential(OrderedDict([
            ('conv2_1', nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn2_1', nn.BatchNorm2d(64)),
            ('leaky2_1', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool2', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer3', nn.Sequential(OrderedDict([
            ('conv3_1', nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn3_1', nn.BatchNorm2d(128)),
            ('leaky3_1', nn.LeakyReLU(0.1, inplace=True)),
            ('conv3_2', nn.Conv2d(128, 64, kernel_size=1, stride=1, padding=0, bias=False)),
            ('bn3_2', nn.BatchNorm2d(64)),
            ('leaky3_2', nn.LeakyReLU(0.1, inplace=True)),
            ('conv3_3', nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn3_3', nn.BatchNorm2d(128)),
            ('leaky3_3', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool3', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer4', nn.Sequential(OrderedDict([
            ('conv4_1', nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn4_1', nn.BatchNorm2d(256)),
            ('leaky4_1', nn.LeakyReLU(0.1, inplace=True)),
            ('conv4_2', nn.Conv2d(256, 128, kernel_size=1, stride=1, padding=0, bias=False)),
            ('bn4_2', nn.BatchNorm2d(128)),
            ('leaky4_2', nn.LeakyReLU(0.1, inplace=True)),
            ('conv4_3', nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn4_3', nn.BatchNorm2d(256)),
            ('leaky4_3', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool4', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer5', nn.Sequential(OrderedDict([
            ('conv5_1', nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn5_1', nn.BatchNorm2d(512)),
            ('leaky5_1', nn.LeakyReLU(0.1, inplace=True)),
            ('conv5_2', nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=0, bias=False)),
            ('bn5_2', nn.BatchNorm2d(256)),
            ('leaky5_2', nn.LeakyReLU(0.1, inplace=True)),
            ('conv5_3', nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn5_3', nn.BatchNorm2d(512)),
            ('leaky5_3', nn.LeakyReLU(0.1, inplace=True)),
            ('conv5_4', nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=1, bias=False)),
            ('bn5_4', nn.BatchNorm2d(256)),
            ('leaky5_4', nn.LeakyReLU(0.1, inplace=True)),
            ('conv5_5', nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn5_5', nn.BatchNorm2d(512)),
            ('leaky5_5', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool5', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer6', nn.Sequential(OrderedDict([
            ('conv6_1', nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn6_1', nn.BatchNorm2d(1024)),
            ('leaky6_1', nn.LeakyReLU(0.1, inplace=True)),
            ('conv6_2', nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=0, bias=False)),
            ('bn6_2', nn.BatchNorm2d(512)),
            ('leaky6_2', nn.LeakyReLU(0.1, inplace=True)),
            ('conv6_3', nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn6_3', nn.BatchNorm2d(1024)),
            ('leaky6_3', nn.LeakyReLU(0.1, inplace=True)),
            ('conv6_4', nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=1, bias=False)),
            ('bn6_4', nn.BatchNorm2d(512)),
            ('leaky6_4', nn.LeakyReLU(0.1, inplace=True)),
            ('conv6_5', nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn6_5', nn.BatchNorm2d(1024)),
            ('leaky6_5', nn.LeakyReLU(0.1, inplace=True))
        ])))
        ]))

        self.classifier = nn.Sequential(OrderedDict([
        ('conv7_1', nn.Conv2d(1024, 1000, kernel_size=(1, 1), stride=(1, 1))),
        ('globalavgpool', GlobalAvgPool2d()),
        ('softmax', nn.Softmax(dim=1))
        ]))

        if pretrained:
            self.load_state_dict(model_zoo.load_url(model_paths['darknet19'],  progress=True))
            print('Model is loaded')

    def forward(self, x):
        out = self.features(x)
        out = self.classifier(out)
        return out

base_model = Darknet19(weights='imagenet', input_shape=(224,224,3))
model_feat = Model(inputs=base_model.input,outputs=predictions)

dark_train_features = model_feat.predict(x_train)
dark_test_features=model_feat.predict(x_test)

    
base_model = tf.keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224,224,3),
    pooling=None
)

X_feat_out = base_model.output
X_feat_flatten = Flatten()(X_feat_out)

X_feat_model = Model(inputs = base_model.input,outputs = X_feat_flatten)
incep_train_features = X_feat_model.predict(X_train)
incep_test_features = X_feat_model.predict(X_test)

base_model = ResNet101(include_top=False, weights='imagenet', input_shape=(224,224,3))
x = base_model.output
x =  Dropout(0.5)(x)
x = Flatten()(x)
x = BatchNormalization()(x)
x = Dense(1024,kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x =  Dropout(0.5)(x)
x = Dense(1024,kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x =  Dropout(0.5)(x)
x = Dense(1024,kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x =  Dropout(0.5)(x)

predictions = Dense(4, activation='softmax')(x)

model_feat = Model(inputs=base_model.input,outputs=predictions)

res_train_features = model_feat.predict(x_train)
res_test_features=model_feat.predict(x_test)

X_train = Concatenate([res_train_features, incep_train_features, dark_train_features])
X_test = Concatenate([res_test_features, incep_test_features, dark_test_features])
import sklearn_relief as sr
r = sr.RReliefF(n_features = 146)
print(r.fit_transform(X_train,y_train))

knn.fit(X_train, y_train) 
print(knn.predict(X_test)) 

In [ ]:
def classifier_summary(pipeline, X_train, y_train, X_test,y_test):
    kidney_fit = pipeline.fit(X_train, y_train)
    y_pred_train= kidney_fit.predict(X_train)
    y_pred_test = kidney_fit.predict(X_test)
    test_accuracy = np.round(accuracy_score(y_test, y_pred_test),4)*100
    test_precision = np.round(precision_score(y_test, y_pred_test, average='weighted'),4)
    test_recall = np.round(recall_score(y_test, y_pred_test, average='weighted'),4)
    test_F1 = np.round(f1_score(y_test, y_pred_test, average='weighted'),4)
    test_kappa =  np.round(cohen_kappa_score(y_test, y_pred_test),4) 
    test_confusion_matrix = confusion_matrix(y_test,y_pred_test)
    print("accuracy : {}%".format(test_accuracy))
    print("F1_score : {}".format(test_F1))
    print("Cohen Kappa Score : {} ".format(test_kappa))
    print("Recall : {}".format(test_recall))
    print("Precision : {}".format(test_precision))
    print("Confusion Matrix : {}".format(test_confusion_matrix))